In [1]:
from progressivis_nb_widgets import *

In [2]:
from progressivis import Scheduler, Every#, log_level
from progressivis.io import CSVLoader
from progressivis.stats.ppca import PPCA
from progressivis.core import aio
from progressivis.stats.blobs_table import BlobsPTable
from progressivis.vis import MCScatterPlot
from progressivis.utils.psdict import PDict
import numpy as np
import pandas as pd

async def moderator(a, b):
    await aio.sleep(0.05)

try:
    s = scheduler
except NameError:
    s = Scheduler()

n_samples = 10_000_000
n_components = 2
rtol = 0.01
centers = [(0.1, 0.3, 0.5), (0.7, 0.5, 3.3), (-0.4, -0.3, -11.1)]
data = BlobsPTable(columns=['_0', '_1', '_2'],  centers=centers,
                          cluster_std=0.2, rows=n_samples, scheduler=s)
ppca = PPCA(scheduler=s)
ppca.input[0] = data.output.result
ppca.params.n_components = n_components
ppca.create_dependent_modules(rtol=rtol)
cnames = ['samples', 'prev_samples']
classes = []
for cn in cnames:
    classes.append({'name': cn, 'x_column': '_pc0',
                    'y_column': '_pc1', 'sample': ppca.reduced, 'sample_slot': cn,
                    'input_module': ppca.reduced, 'input_slot': 'result'})
sp = MCScatterPlot(scheduler=s, classes=classes)
sp.create_dependent_modules()
for cn in cnames:
    sp[cn].min_value.result.update({'_pc0': -np.inf, '_pc1': -np.inf})
    sp[cn].max_value.result.update({'_pc0': np.inf, '_pc1': np.inf})
sp_widget = Scatterplot()
sp_widget.link_module(sp)
s.task_start(tick_proc=moderator);

In [3]:
display(sp_widget)

Scatterplot(to_hide=[])

Starting scheduler
# Scheduler added module(s): ['blobs_table_1', 'dyn_var_1', 'dyn_var_2', 'histogram_index_1', 'histogram_index_2', 'histogram_index_3', 'histogram_index_4', 'max_1', 'max_2', 'max_3', 'max_4', 'mc_histogram2_d_1', 'mc_histogram2_d_2', 'mc_scatter_plot_1', 'merge_dict_1', 'merge_dict_2', 'merge_dict_3', 'merge_dict_4', 'min_1', 'min_2', 'min_3', 'min_4', 'ppca_1', 'ppca_transformer_1', 'range_query2d_1', 'range_query2d_2', 'sample_1']
